In [ ]:
import math
import pandas as pd

In [ ]:
MODEL_ORDER = ["small", "medium", "large", "xl", "2.7B"]
df = pd.read_csv("raw_data/benchmark_results_comp.csv")
df = df.rename(columns={"model.context_length": "Context", "model": "Model", "forward_mean": "forward", "backward_mean": "backward", "optimizer_mean": "optimizer"})\
    [["compile", "forward_only", "Model", "Context", "forward", "backward", "optimizer"]]
df["Model"] = pd.Categorical(df["Model"], categories=MODEL_ORDER, ordered=True)

In [ ]:
df

In [ ]:
def prepare_report(df1, col):
    eager = df1[~df1.compile].pivot_table(
        index="Model", columns="Context", values=col, observed=False,
    )
    compiled = df1[df1.compile].pivot_table(
        index="Model", columns="Context", values=col, observed=False,
    )

    # Align and combine into the formatted string
    report = eager.combine(
        compiled,
        lambda a, b: a.combine(
        b,
        lambda x, y: f"{x*1000:.1f} / {y*1000:.1f}" if pd.notna(x) and pd.notna(y)
                    else f"{x*1000:.1f} / OOM" if pd.notna(x)
                    else f"OOM / {y*1000:.1f}" if pd.notna(y)
                    else "OOM / OOM"
       ),
    )

    return report.sort_index()

In [ ]:
forward_infer = prepare_report(df[df.forward_only], "forward")
forward_train = prepare_report(df[~df.forward_only], "forward")
backward = prepare_report(df[~df.forward_only], "backward")
optimizer = prepare_report(df[~df.forward_only], "optimizer")

In [ ]:
forward_infer

In [ ]:
backward

In [ ]:
optimizer

In [ ]:
def print_report(df, title):
    tex = df.to_latex(index=True, caption=f"{title} time in ms (eager / compiled)", escape=False)
    tex = tex.replace(r"\begin{table}", r"\begin{table}[H]")
    print(tex)

In [ ]:
print_report(forward_infer, "Forward inference")
print_report(forward_train, "Forward train")
print_report(backward, "Backward")
print_report(optimizer, "Optimizer")